In [1]:
import pandas as pd

In [17]:
df = pd.read_csv("../Datasets/01-census-income.csv")
df2 = pd.read_csv("../Datasets/02-future-census.csv")

In [5]:
cols = ['age','edu num', 'marital status','occupation',
'sex','captial-gain','capital-loss','hours-per-week']

In [19]:
X_train = df[cols]
X_test = df2[cols]
y_train = df.label
y_test = df2.label

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer

In [8]:
transformer = make_column_transformer(
    (OneHotEncoder(),[ 'marital status','occupation',
'sex']),
    (StandardScaler(),['age','edu num','captial-gain','capital-loss','hours-per-week'])
)

In [10]:
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

# Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression

In [24]:
logreg = LogisticRegression(max_iter=1000,warm_start=True)
logreg.fit(X_train_transformed,y_train)

LogisticRegression(max_iter=1000, warm_start=True)

In [25]:
logreg.score(X_train_transformed,y_train)

0.8496486981165495

In [26]:
logreg.score(X_test_transformed,y_test)

0.8506347256347256

In [28]:
logreg.decision_function(X_test_transformed[5:10])

array([-1.59148656,  1.23995133, -1.80480941, -0.21699711,  0.3503315 ])

# SVM

In [29]:
from sklearn.svm import LinearSVC

In [31]:
lin_svc = LinearSVC(max_iter=5000)
lin_svc.fit(X_train_transformed,y_train)

LinearSVC(max_iter=5000)

In [32]:
lin_svc.score(X_train_transformed,y_train)

0.8491763594497255

In [33]:
lin_svc.score(X_test_transformed,y_test)

0.8520679770679771

In [34]:
lin_svc.coef_

array([[-0.2858367 ,  0.63418974,  0.41900813, -0.25818619, -0.40039113,
        -0.25665247, -0.2838421 , -0.24461596,  0.07721664,  0.35239988,
        -0.02234751,  0.29195469, -0.43637988, -0.25474134, -0.13444195,
        -0.28340754, -0.38530357,  0.19692807,  0.16493982,  0.10657722,
         0.20971436, -0.07020365, -0.24529807, -0.18641265,  0.10591116,
         0.24075282,  0.76208055,  0.09159255,  0.11826131]])

In [35]:
lin_svc.intercept_

array([-0.43171072])

In [36]:
lin_svc.decision_function(X_test_transformed)

array([-0.8276655 ,  0.60990266, -1.8626276 , ...,  0.19100426,
       -0.59874335, -0.76500912])

In [37]:
from sklearn.svm import SVC ##ควรทำCV

In [38]:
svc = SVC()

In [39]:
svc_poly = SVC(kernel="poly")
svc_poly.fit(X_train_transformed,y_train)

SVC(kernel='poly')

In [40]:
svc_poly.score(X_train_transformed,y_train)

0.8571175532857058

In [41]:
svc_poly.score(X_test_transformed,y_test)

0.8530917280917281

In [42]:
svc_rbf =SVC(kernel='rbf')
svc_rbf.fit(X_train_transformed,y_train)

SVC()

In [44]:
svc_rbf.score(X_train_transformed,y_train), svc_rbf.score(X_test_transformed,y_test)

(0.8587412174529138, 0.8552416052416052)

# Ensammble

In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB

# Voting -- Need to get parameter by grid CV

In [72]:
voting = VotingClassifier(estimators=[
    ('knn', KNeighborsClassifier(35)),
    ('dtree', DecisionTreeClassifier(max_depth=10)),
    ('logistic', LogisticRegression(max_iter=1000)),
    ('svm', SVC(kernel='poly')),
    ('gnb',GaussianNB())
])

In [73]:
voting.fit(X_train_transformed.toarray(),y_train)

VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=35)),
                             ('dtree', DecisionTreeClassifier(max_depth=10)),
                             ('logistic', LogisticRegression(max_iter=1000)),
                             ('svm', SVC(kernel='poly')),
                             ('gnb', GaussianNB())])

In [74]:
voting.score(X_train_transformed.toarray(),y_train)

0.8596268524532089

In [75]:
voting.score(X_test_transformed.toarray(),y_test)

0.8531941031941032

# Bagging -- No need CV

In [76]:
from sklearn.ensemble import BaggingClassifier

In [81]:
bag = BaggingClassifier(KNeighborsClassifier(35), max_samples=1000,
n_estimators=100,oob_score=True)

In [82]:
bag.fit(X_train_transformed,y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=35),
                  max_samples=1000, n_estimators=100, oob_score=True)

In [83]:
bag.oob_score_

0.8350061994450021

In [84]:
bag.score(X_train_transformed,y_train)

0.8352128476117376

In [86]:
bag.score(X_test_transformed,y_test)

0.8337428337428338

# Random Forest - ไม่จำเป็น CV but It can use

In [85]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
forest = RandomForestClassifier(n_estimators=1000,max_depth=2,
oob_score=True)

In [88]:
forest.fit(X_train_transformed,y_train)

RandomForestClassifier(max_depth=2, n_estimators=1000, oob_score=True)

In [89]:
forest.oob_score_ ##เหมือนคะแนนเฉลี่ยของ Cross validate

0.7914034362638012

In [90]:
forest.score(X_train_transformed,y_train)

0.7905473224301824

In [92]:
forest.score(X_test_transformed,y_test)

0.7848075348075348

## Find Best para on Random by CV

In [93]:
from sklearn.model_selection import GridSearchCV

In [94]:
parameter_grid = {'n_estimators':[100,500,1000],
'max_depth':[1,2,3,4,5]}

In [95]:
grid_search = GridSearchCV(RandomForestClassifier(),parameter_grid,
cv=5)

In [96]:
grid_search.fit(X_train_transformed,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [1, 2, 3, 4, 5],
                         'n_estimators': [100, 500, 1000]})

In [97]:
grid_search.best_score_

0.8500029742547204

In [98]:
grid_search.score(X_train_transformed,y_train)

0.8504457696168153

In [99]:
grid_search.score(X_test_transformed,y_test)

0.8456183456183456

In [100]:
grid_search.best_estimator_.feature_importances_

array([9.44342962e-03, 2.25807441e-04, 2.67001721e-01, 2.45461343e-04,
       9.46946998e-02, 1.77154493e-03, 1.07937416e-03, 9.30129864e-04,
       6.55363429e-04, 2.58460208e-06, 4.50338735e-04, 2.74262590e-02,
       1.71138862e-03, 2.55101971e-03, 1.25192811e-03, 9.48339296e-03,
       1.10226357e-05, 2.30918188e-02, 7.62570057e-05, 2.24485226e-04,
       2.91852580e-04, 9.51145647e-04, 2.45343244e-02, 2.70204382e-02,
       7.32603576e-02, 1.57025934e-01, 1.91796668e-01, 3.73123463e-02,
       4.54789060e-02])

In [102]:
grid_search.best_params_

{'max_depth': 5, 'n_estimators': 100}

# AdaBoostClassifier - No need CV

In [103]:
from sklearn.ensemble import AdaBoostClassifier

In [104]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
n_estimators=30, learning_rate=0.5)

In [105]:
ada.fit(X_train_transformed,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30)

In [106]:
ada.estimator_errors_ ##Error เพิ่มขึ้นเรื่อยๆ

array([0.2387672 , 0.28116323, 0.34761989, 0.4077288 , 0.37117376,
       0.38784679, 0.39455626, 0.45763459, 0.4660478 , 0.45650082,
       0.44524475, 0.45486772, 0.44633659, 0.43431209, 0.44122301,
       0.45755603, 0.47636744, 0.48634819, 0.4819332 , 0.4746979 ,
       0.48527814, 0.49038798, 0.48032889, 0.47350035, 0.48886505,
       0.48736257, 0.47551487, 0.4947119 , 0.48414754, 0.49630563])

In [107]:
ada.estimator_weights_ ## นน เท่ากัน

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [108]:
ada.score(X_train_transformed,y_train)

0.8560252701186751

In [109]:
ada.score(X_test_transformed,y_test)

0.8565724815724816

# Gradient Boosting

In [110]:
from sklearn.ensemble import GradientBoostingClassifier

In [111]:
gbc = GradientBoostingClassifier()

In [112]:
gbc.fit(X_train_transformed,y_train)

GradientBoostingClassifier()

In [113]:
gbc.score(X_train_transformed,y_train)

0.8679813426226605

In [114]:
gbc.score(X_test_transformed,y_test)

0.8648648648648649

# Stack

In [115]:
from sklearn.ensemble import StackingClassifier

In [117]:
stacking = StackingClassifier(estimators=[('knn', KNeighborsClassifier(35)),
    ('dtree', DecisionTreeClassifier(max_depth=10)),
    ('logistic', LogisticRegression(max_iter=1000)),
    ('svm', SVC(kernel='poly')),
    ('gnb',GaussianNB()),
    ],
     final_estimator= RandomForestClassifier()
    )

In [120]:
stacking.fit(X_train_transformed.toarray(),y_train)

StackingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=35)),
                               ('dtree', DecisionTreeClassifier(max_depth=10)),
                               ('logistic', LogisticRegression(max_iter=1000)),
                               ('svm', SVC(kernel='poly')),
                               ('gnb', GaussianNB())],
                   final_estimator=RandomForestClassifier())